### Pulling SNOTEL Data via EasySNowData

In [7]:
%load_ext autoreload
%autoreload 2
import easysnowdata as esd
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import tqdm
import contextily as ctx
import xarray as xr
import glob
import requests 
from io import StringIO

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
StationCollection = esd.automatic_weather_stations.StationCollection()

Geodataframe with all stations has been added to the Station object. Please use the .all_stations attribute to access.
Use the .get_data(stations=geodataframe/string/list,variables=string/list,start_date=str,end_date=str) method to fetch data for specific stations and variables.


In [11]:
all_snotel = StationCollection.all_stations[StationCollection.all_stations.network == 'SNOTEL']

In [13]:
all_snotel.explore()

In [19]:
# mow, let's also get our grid data to look for overlaps
tile_gfs = []
utm_zones = ['utm10n', 'utm11n', 'utm12n', 'utm13n']
home_path = r"C:\Users\JackE\uw\courses\aut24\ml_geo\final_data"
for utm_zone in utm_zones:
    aso_paths = glob.glob(rf'{home_path}\ASO_50m_SD_cleaned\{utm_zone}\*')
    tile_names = [f'train_{utm_zone}_32km.shp', f'test_{utm_zone}_32km.shp', f'val_{utm_zone}_32km.shp']
    for tile_set in tile_names:
        # change all to 4326 for concat purposes
        tiles = gpd.read_file(rf'{home_path}\tiles\{tile_set}').to_crs(all_snotel.crs)
        tile_gfs.append(tiles)

In [21]:
gf_tiles = pd.concat(tile_gfs).reset_index(drop=True)

In [23]:
snotel_tiles = all_snotel.clip(gf_tiles)

In [25]:
m = gf_tiles.explore(color='black', alpha=0.3)
snotel_tiles.explore(m=m)

In [26]:
len(snotel_tiles)

136

In [27]:
# now let's peak into what this data looks like
# first, we'll have the get the station's code from the df below
snotel_tiles.head()

,name,network,elevation_m,latitude,longitude,state,HUC,mgrs,mountainRange,beginDate,endDate,csvData,geometry
code,,,,,,,,,,,,,
580_CO_SNTL,Lily Pond,SNOTEL,3352.800049,37.379292,-106.548347,Colorado,130100020302,13SCB,Southern Rocky Mountains,1979-10-01,2024-10-23,True,POINT (-106.54835 37.37929)
1058_CO_SNTL,Grayback,SNOTEL,3541.775879,37.470329,-106.537827,Colorado,130100020101,13SCB,Southern Rocky Mountains,2004-09-01,2024-10-23,True,POINT (-106.53783 37.47033)
797_CO_SNTL,Stump Lakes,SNOTEL,3413.760010,37.476212,-107.632950,Colorado,140801040702,13SBB,Southern Rocky Mountains,1985-10-01,2024-10-23,True,POINT (-107.63295 37.47621)
874_CO_SNTL,Wolf Creek Summit,SNOTEL,3352.800049,37.479221,-106.801697,Colorado,140801010203,13SCB,Southern Rocky Mountains,1985-10-01,2024-10-23,True,POINT (-106.8017 37.47922)
905_CO_SNTL,Mancos,SNOTEL,3048.000000,37.430870,-108.169540,Colorado,140801070102,12SYG,Southern Rocky Mountains,1994-08-30,2024-10-23,True,POINT (-108.16954 37.43087)


In [44]:
# now we can get our daily snowpack point data we want to encode
test_station = esd.automatic_weather_stations.StationCollection()
test_station.get_data("580_CO_SNTL")
test_station.data

Geodataframe with all stations has been added to the Station object. Please use the .all_stations attribute to access.
Use the .get_data(stations=geodataframe/string/list,variables=string/list,start_date=str,end_date=str) method to fetch data for specific stations and variables.
Only one station chosen with variables=None. Default behavior fetches all variables for this station.
Dataframe has been added to the Station object. Please use the .data attribute to access.


,TAVG,TMIN,TMAX,SNWD,WTEQ,PRCPSA
datetime,,,,,,
1980-07-11,0.0,0.0,0.0,NaN,NaN,NaN
1980-07-12,0.0,0.0,0.0,NaN,NaN,NaN
1980-07-13,0.0,0.0,0.0,NaN,NaN,NaN
1980-07-14,0.0,0.0,0.0,NaN,NaN,NaN
1980-07-15,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...
2024-10-19,1.0,-1.2,5.0,0.1524,0.0432,0.0178
2024-10-20,0.6,-3.5,6.8,0.2286,0.0559,0.0076
2024-10-21,1.2,-4.3,8.9,0.2032,0.0635,0.0000
